# Diffusion math insights
This is the unfiltered bullet point list of miscellaneous insights for diffusion math.

* Diffusion is easier to train because each step only small variance and mean change, so the gaussian doesn't change too much. Much harder to train with just one step from noise to image.
* forward process => transforms the data from in-distribution to out-distribution
* backward process => move back from out-distribution to in-distribution
* forward => training, backward => sampling (inference)
* dual interpretation of forward process 1. gaussian shift + scaling 2. scaling image down + adding noise to it
* new batch is used each iteration
* timestep is sampled from uniform distribution - probably has to do with monte carlo
* format of x data, in DDPM and stable diffusion it's 2D matrix for the U-net, but in stable diffusion they use VAE to reduce the dimensionality.
* possible formulation where you can let a network learn the SNR that is the ratio of signal and noise. This constitutes the scale weighting outside the mean squared error in the objective. So you let the network learn the scheduling variance basically
* since all of the steps in the forward process are normal noise additions with mean 0 and different variances we can just use the properties of the normal distribution to create inputs to the diffusion model for any timestep in a single step, rather than having to do a bunch of incremental noise additions in a loop 
but if you did do the forward process by doing a bunch of incremental noise additions you would then train the diffusion model to predict the sum of the noise increments you sampled, suitably rescaled so that its expected variance was 1 instead of depending on however many increments you summed
* U-net is learning the total noise summed from t=0 to t=T scaled to N(0, I) regardless of timestep.
* For gaussian distributions covariance zero means independence, so each entry in the noise 2D matrix can be seen as having their own univariate gaussian distribution, although the noise is sampled from a multivariate gaussian
* mean prediction formulation not as good, because the mean will not change that much in the forward process?
* Forward process can be seen as a form of data augmentation, because we will create new data each time a stochastic noise is added to it
* Unsupervised since we do not have labels in the training data, the noise from sampled from gaussian is our labels, so we generate them during training
* backward process when doing sampling everytime is seen as a decision point where you can go from one mode to another (seems to align with score-based formulation)
* Once training data is perturbed by noise it turns the very complicated data distribution with many many modes into a gaussian mixture with lots of modes? Although gaussian noise is multivariate so it shouldn't have multi modes?
* Bigger batches better according to Kat, because gradients is really noisy and bigger batches will help combat that
* The scaling weight factor outside the MSE is dropped in DDPM and SD, lead to better training
* mean moves towards zero while variance moves towards one, so we finally get N(0, I)
* 	• start with maximize log likelihood log(p_theta(x_0)) by marginalizing the latents, but this is intractable
	• Find ELBO and maximize it
	• Equivalent to minimizing the KL divergence between the reverse process 
D_KL(q(x_t-1|x_t,x_0)  || p_theta(x_t-1 | x_t)), where p_theta is our model and q is our true function
* Diffusion paths, lots of possible latents can pass through a point in the diffusion path
* diffusion backward scheduler only takes a small proportion of the predicted noise, so we can move slowly from T to t=0. Each timestep will produce a latent that is slowly denoised
* Forward process continuous values. Reverse process continuous but has to convert to discrete before reaching the image x_0, because pixels in an image are discrete, not continuous
* implicit loss weightings depending on objective (mean of posterior reverse, noise formulation, predict original image x_0 given arbitrary latent x_t, learn score function formulation)
* in understanding diffusion models: a unified perspective, the reconstruction term $p(x_0|x_1)$ and the prior $p(x_T|x_0)$ is ignored because the beta in a scheduler can be made so that the very first step doesn't introduce so much noise, so it doesn't affect the original image and the last step introduces as much noise possible to make the mean go to zero and variance go to 1. So for the reconstruction term we can ignore it because it doesn't change that much at all, whereas for the last step because we will try to make it converge to a isotropic gaussian (zero mean and diagonal covariance matrix) we can just approximate that one with the prior distribution that we already know.
* for the noise schedule we have $\alpha_t = \Pi_{t=1}^{T}(1-\beta_t)$. Now it depends on what scheduler we use so the behavior will vary. Nevertheless, in general $\alpha_t \to 0$, $1-\alpha_t \to 1$ and $\beta_t \to 1$ as $t \to T$, where $\beta_t$ starts from 0
* By maximizing ELBO, theoretically we could get it to the same as the log marginal, that is the same as the data distribution. But in reality we are never that lucky. Still this means that as we are maximizing the ELBO we are moving closer to making a better approximate of the data distribution as well. That's why in the end we can use our trained model to generate samples that reflects the data distribution. I think this is an indirect MLE happening, because if ELBO is maximized to the point that KL-divergence is zero, then the log marginal or log evidence p(x) must be as high as possible, which is what MLE would try to achieve.
* VAE reduces dimensionality, this makes it faster and easier to train in lower dimensionality latents. And then upscales the image during inference. U-net learns the noise in a given noisy image latent
* Sampler and scheduler is different! Some people use them interchangeably but they are not the same thing!
* There is one formula 93 and 94 in the 2022 paper understanding diffusion models: a unified perspective, where reparametrizing $x_0$ will give different objectives.
* 


interpretations of the objective
1. learn mean of the gaussian distribution of the backward process $p_{\theta}(x_{t-1}|x_t)$
2. learn to predict original image $x_0$ given an arbitrary noised image $z_t$
3. learn the noise added to image
4. learn score-based